# Pipeline 4: Identifying abnormal laboratory results

This notebook presents all code used for the evaluation of the code produced for the fourth pipeline, as well as the related relational algebra

# Relational algebra
$D_{labevents}(labevent\_id, subject\_id, hadm\_id, specimen\_id, itemid, order\_provider\_id, charttime, storetime, value, valuenum, valueuom, ref\_range\_lower, ref\_range\_upper, flag, priority, comments)$

$labevents\_abnormal = \sigma_{flag='abnormal'}(labevents)$

$labevents\_abnormal\_deviate = \pi_{*, deviation\_from\_range(*)}(labevents\_abnormal)$

$labevents\_abnormal\_deviate = \rho_{patient\_id/subject\_id}(labevents\_abnormal\_deviate)$

$final\_data = \pi_{labevent\_id, patient\_id, hadm\_id, specimen\_id, itemid, charttime, storetime, value, valuenum, valueuom, ref\_range\_lower, ref\_range\_upper, deviation\_from_range}(abevents\_abnormal\_deviate)$

In [ ]:
# Necessary imports and setup of connection to the database

import polars as pl
from sqlalchemy import create_engine, inspect, Table, MetaData, Column, Integer, String, DateTime, Float, Boolean
import numpy as np

engine = create_engine(r"sqlite://path to where mimic4.db is stored") #change this to the path where mimic.db is stored

In [ ]:
# Ground truth code, used for comparison
metadata = MetaData()

labevents = Table(
    'labevents', metadata,
    Column('labevent_id', Integer, nullable=False),
    Column('subject_id', Integer, nullable=False),
    Column('hadm_id', Integer, primary_key=True),
    Column('specimen_id', Integer, nullable=False),
    Column('itemid', Integer, primary_key=True, nullable=False),
    Column('order_provider_id', String, nullable=True),
    Column('charttime', DateTime, nullable=False),
    Column('storetime', DateTime, nullable=False),
    Column('value', String, nullable=False),
    Column('valuenum', Float, nullable=True),
    Column('valueuom', String, nullable=False),
    Column('ref_range_lower', Float, nullable=False),
    Column('ref_range_upper', Float, nullable=False),
    Column('flag', String, nullable=False),
    Column('priority', String, nullable=True),
    Column('comments', String, nullable=True) 
)

metadata.reflect(bind=engine)

query = """
SELECT *
FROM labevents
WHERE flag = 'abnormal'

"""
df = pl.read_database(query=query, connection=engine.connect(), infer_schema_length=1000)

df.with_columns(pl.col('hadm_id').cast(pl.Int64))

# Calculate the deviation of valuenum from the accepted range
lower_deviation = df['ref_range_lower'] - df['valuenum']
upper_deviation = df['valuenum'] - df['ref_range_upper']

# Create a new column with deviations
df = df.with_columns(
    pl.when(df['valuenum'] < df['ref_range_lower'])
    .then(lower_deviation)
    .when(df['valuenum'] > df['ref_range_upper'])
    .then(upper_deviation)
    .otherwise(0)
    .alias("deviation_from_range")
)

df = df.rename({"subject_id": "patient_id"})

df_gt = df.select(['labevent_id', 'patient_id', 'hadm_id', 'specimen_id', 'itemid',
                        'charttime', 'storetime', 'value', 'valuenum', 'valueuom', 'ref_range_lower', 'ref_range_upper', 'deviation_from_range'])



# Code Valdity 

In the code block below, the generated code produced by the LLM can be pasted and executed, to assess if the code works without any runtime errors.

In [ ]:
# Paste generated code here 

# Code Correctness

To assess the code correctness, the code below can be executed. If it is correct, True should be returned. Otherwise, manual inspection in the code block below should be executed. It is assumed the presented solution is stored in a variable called 'final data'# Code correctness

In [ ]:
df_gt.equals(final_data)

In [ ]:
#  Execute this code if the dataframes are not equal to see the differences
df_combined_pandas = df_combined.to_pandas()
final_data_pandas = final_data.to_pandas()

df_diff = df_combined_pandas.compare(final_data_pandas)

if not df_combined_pandas.equals(final_data_pandas):
    print(df_diff)

# Relative Efficiency

To record the average running time for the generated solution, the generated solution can be added to the code block below and executed. The average executrion time should be recorded per solution, for later determination of the relative efficiency

In [ ]:
import time
import numpy as np
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

execution_times = []

for i in range(10):
    start_time = time.time()

    # Paste generated code here

    end_time = time.time()

    execution_time = end_time - start_time
    execution_times.append(execution_time)


average_execution_time = np.mean(execution_times)

print(f"The code executed in average {average_execution_time} seconds over 10 runs.")

In [ ]:
# To calculate the relative efficiency of the generated solutions
def relative_efficiencies(times):
    tmin = min(times)
    tmax = max(times)
    efficiencies = [100 * (1 - (tc - tmin) / (tmax - tmin)) for tc in times]
    return efficiencies

time_gt = [365.7] # ground truth time
solutions = [] # Add the times from the 10 runs here
times = time_gt + solutions


re = relative_efficiencies(times)
# Calculate average
average = np.mean(re[1:])
print(f'Average: {average}')